 ## Example 11: How we solve system of eq for element


In [ ]:
# code independent on BG framework here just for explanation purposes


In [1]:
import sympy as sp

# ============================================================
# 1. Helper functions
# ============================================================

def is_ef(sym: sp.Symbol) -> bool:
    """
    Check whether a symbol is of type e* or f*.
    These variables must be recursively unfolded.
    """
    return sym.name[0] in ('e', 'f')


def solve_single(var, eqs, visited):
    """
    Extract all possible expressions of the form:

        var = expression

    from the given equation list.
    This includes:
      • direct equations (Eq(var, expr))
      • reversed equations (Eq(expr, var))
      • implicitly solvable equations (using sympy.solve)

    Returns list of candidate expressions for var.
    """
    candidates = []

    for eq in eqs:
        lhs, rhs = eq.lhs, eq.rhs

        # Direct form: var = RHS
        if lhs == var:
            candidates.append(rhs)
            continue

        # Reversed form: LHS = var  → var = LHS
        if rhs == var:
            candidates.append(lhs)
            continue

        # If var appears anywhere, try solving symbolically
        if var in eq.free_symbols:
            try:
                sols = sp.solve(eq, var, dict=False)
                if sols:
                    candidates.append(sols[0])
            except Exception:
                pass  # not solvable; ignore

    return candidates


def expand_expr(expr, eqs, visited):
    """
    Recursively expand all e* and f* symbols inside an expression.
    Parameters inside the expression remain untouched.

    Example:
        If expr = e4 + e9 - SE1
        then it recursively unfolds e4 and e9.
    """
    new_expr = expr

    for s in list(expr.free_symbols):
        if is_ef(s):
            # Recursively compute substitution for s
            sub = solve_for(s, eqs, visited)
            new_expr = sp.simplify(new_expr.subs(s, sub))

    return sp.simplify(new_expr)


# ============================================================
# 2. Main recursive solver function
# ============================================================

def solve_for(var, eqs, visited=None):
    """
    Recursively solves the target variable 'var'
    entirely in terms of *parameters only*.

    e.g., solve_for(e8, eqs) should return an expression
    with NO e* or f* variables inside.

    This function:
      • finds all possible expressions of var
      • chooses the "simplest" one (fewest EF symbols)
      • recursively expands dependencies
      • detects cycles
    """

    # If var is not an e* or f* → it is a parameter → return as-is
    if not is_ef(var):
        return var

    # Initialize cycle-detection set
    if visited is None:
        visited = set()

    # Cycle detected
    if var in visited:
        raise RuntimeError(f"Cycle while expanding {var}")

    # Add this variable to visited chain
    visited = visited | {var}

    # Get all possible var = expr candidates
    candidates = solve_single(var, eqs, visited)
    if not candidates:
        raise RuntimeError(f"Variable {var} cannot be solved from given equations.")

    # Rank candidates:
    # Fewer (e*, f*) inside expression → higher priority
    scored = []
    for expr in candidates:
        ef_vars = [s for s in expr.free_symbols if is_ef(s)]
        score = len(ef_vars)
        scored.append((score, expr))

    scored.sort(key=lambda x: x[0])

    # Try each candidate until one succeeds
    last_err = None
    for score, expr in scored:
        try:
            result = expand_expr(expr, eqs, visited)
            return result  # success
        except RuntimeError as e:
            last_err = e
            continue

    # If all candidates failed, throw last error
    if last_err:
        raise last_err

    raise RuntimeError(f"Cannot expand {var}")



In [2]:
# ============================================================
# 3. Example test function using your exact equation set
# ============================================================

# Declare variables again inside this scope
e = {i: sp.symbols(f"e{i}") for i in range(20)}
f = {i: sp.symbols(f"f{i}") for i in range(20)}

# Parameters
SE0, SE1 = sp.symbols("SE0 SE1")
p7, I7, p8, I8 = sp.symbols("p7 I7 p8 I8")
q3, C3, q4, C4 = sp.symbols("q3 C3 q4 C4")
R5, R6, SF2 = sp.symbols("R5 R6 SF2")

# ----- YOUR EQUATIONS HERE -----
eqs = [
    sp.Eq(e[0], SE0),
    sp.Eq(e[1], SE1),
    sp.Eq(f[2], p7/I7),
    sp.Eq(e[6], q3/C3),
    sp.Eq(e[7], R5*f[7]),
    sp.Eq(f[8], p8/I8),
    sp.Eq(e[11], q4/C4),
    sp.Eq(e[12], R6*f[12]),
    sp.Eq(f[14], SF2),
    sp.Eq(f[2], f[0]),
    sp.Eq(f[3], f[0]),
    sp.Eq(e[0] - e[2] - e[3], 0),
    sp.Eq(f[4], f[1]),
    sp.Eq(f[8], f[1]),
    sp.Eq(f[9], f[1]),
    sp.Eq(e[1] - e[4] - e[8] - e[9], 0),
    sp.Eq(e[4], e[3]),
    sp.Eq(e[5], e[3]),
    sp.Eq(f[3] + f[4] + f[5], 0),
    sp.Eq(f[6], f[5]),
    sp.Eq(f[7], f[5]),
    sp.Eq(e[5] - e[6] - e[7], 0),
    sp.Eq(e[10], e[9]),
    sp.Eq(e[13], e[9]),
    sp.Eq(f[10] + f[13] + f[9], 0),
    sp.Eq(f[11], f[10]),
    sp.Eq(f[12], f[10]),
    sp.Eq(e[10] - e[11] - e[12], 0),
    sp.Eq(f[14], f[13]),
    sp.Eq(e[13] - e[14], 0)
]

# Solve e8 fully
result = solve_for(e[8], eqs)
print("Fully expanded e8 =")
print(sp.simplify(result))


Fully expanded e8 =
R6*SF2 + SE1 + R5*p8/I8 + R6*p8/I8 + R5*p7/I7 - q4/C4 - q3/C3


In [11]:
eqs

[Eq(e0, SE0),
 Eq(e1, SE1),
 Eq(f2, p7/I7),
 Eq(e6, q3/C3),
 Eq(e7, R5*f7),
 Eq(f8, p8/I8),
 Eq(e11, q4/C4),
 Eq(e12, R6*f12),
 Eq(f14, SF2),
 Eq(f2, f0),
 Eq(f3, f0),
 Eq(e0 - e2 - e3, 0),
 Eq(f4, f1),
 Eq(f8, f1),
 Eq(f9, f1),
 Eq(e1 - e4 - e8 - e9, 0),
 Eq(e4, e3),
 Eq(e5, e3),
 Eq(f3 + f4 + f5, 0),
 Eq(f6, f5),
 Eq(f7, f5),
 Eq(e5 - e6 - e7, 0),
 Eq(e10, e9),
 Eq(e13, e9),
 Eq(f10 + f13 + f9, 0),
 Eq(f11, f10),
 Eq(f12, f10),
 Eq(e10 - e11 - e12, 0),
 Eq(f14, f13),
 Eq(e13 - e14, 0)]